In [1]:
import os
import numpy as np
from utils.tools import Tools


# KP Problem

## Knapsack

In [2]:
def is_positive_integer(X):
    return np.logical_and((X > 0), (np.floor(X) == X))

def knapsack(weights, values, W):
    """KNAPSACK Solves the 0-1 knapsack problem for positive integer weights

  [BEST AMOUNT] = KNAPSACK(WEIGHTS, VALUES, CONSTRAINT)
       
       WEIGHTS    : The weight of every item (1-by-N)
       VALUES     : The value of every item (1-by-N)
       CONSTRAINT : The weight constraint of the knapsack (scalar)

       BEST       : Value of best possible knapsack (scalar)
       AMOUNT     : 1-by-N vector specifying the amount to use of each item (0 or 1)


    EXAMPLE :

        weights = [1 1 1 1 2 2 3];
        values  = [1 1 2 3 1 3 5];
        [best amount] = KNAPSACK(weights, values, 7)

        best =

            13


        amount =

             0     0     1     1     0     1     1
"""

    if not all(is_positive_integer(weights)) or not is_positive_integer(W):
        raise Exception('Weights must be positive integers')

    # We work in one dimension
    #     M, N = weights.shape;
    weights = weights[:]
    values = values[:]
    if len(weights) != len(values):
        raise Exception('The size of weights must match the size of values')

    # Solve the problem
    A = np.zeros((len(weights) + 1, W + 1))
    # A(j+1,Y+1) means the value of the best knapsack with capacity Y using
    # the first j items.
    for j in range(len(weights)):
        for Y in range(W):
            if weights[j] > Y + 1:
                A[j + 1, Y + 1] = A[j, Y + 1]
            else:
                A[j + 1, Y + 1] = max(A[j, Y + 1], values[j] + A[j, int(Y - weights[j] + 1)])

    best = A[-1, -1]
    # Now backtrack
    amount = np.zeros(len(weights))
    a = best
    j = len(weights) - 1
    Y = W - 1
    while a > 0:
        while A[j + 1, Y + 1] == a:
            j = j - 1

        j = j + 1  # This item has to be in the knapsack
        amount[j] = 1
        Y = int(Y - weights[j])
        j = j - 1
        a = A[j + 1, Y + 1]
    return best, amount

In [3]:
weights = np.array((1 , 1, 1, 1, 2, 2, 3))
values  = np.array((1 , 1, 3, 2, 1, 3, 5))
best, amount = knapsack(weights, values, 1)
print(best)
print(amount)

3.0
[0. 0. 1. 0. 0. 0. 0.]


In [4]:
def GenKnapsack(n=1000, v=10, r=5, type_wp='uc', type_c='rk', addr="problems/knapsack"):
    assert type_wp in ['uc', 'wc', 'sc'], 'type_wp is not valid'
    assert type_c in ['rk', 'ak'], 'type_wp is not valid'
    # type_wc: Các trường hợp phân loại KP 
    #     *uc : không phù hợp
    #     *wc : phù hợp mức độ thấp
    #     *sc : phù hợp cao 
    # type_c: Công suất chứa 
    #     *rk : Công suất hạn chế
    #     *ak : Công suất trung bình
    w = (1 + np.round(np.random.rand(n) * (v - 1)))
    if type_wp == 'uc':
        p = 1 + np.round(np.random.rand(n) * (v - 1))
    elif type_wp == 'wc':
        p = w + np.round(r - 2 * r * np.random.rand(n))
        p[p <= 0] = w[p <= 0]
    elif type_wp == 'sc':
        p = w + r

    if type_c == 'rk':
        cap = int(2 * v)
    elif type_c == 'ak':
        cap = int(0.5 * np.sum(w))

    th_best, _ = knapsack(w, p, cap)

    KP_uc_rk = {}
    KP_uc_rk['w'] = w
    KP_uc_rk['p'] = p
    KP_uc_rk['cap'] = cap
    KP_uc_rk['opt'] = th_best

    Tools.save_to_file(os.path.join(addr, 'KP_{}_{}'.format(type_wp, type_c)), KP_uc_rk)

In [5]:
knapsack_problem_path = 'problems/knapsack'
for type_wp in ['uc', 'wc', 'sc']:
    for type_c in ['rk', 'ak']:
        GenKnapsack(type_wp=type_wp, type_c=type_c, addr=knapsack_problem_path)

In [8]:
knapsack_problem_path = 'problems/knapsack'

KP_sc_ak = Tools.load_from_file(os.path.join(knapsack_problem_path, 'KP_sc_ak'))
KP_uc_ak = Tools.load_from_file(os.path.join(knapsack_problem_path, 'KP_uc_ak'))
KP_wc_ak = Tools.load_from_file(os.path.join(knapsack_problem_path, 'KP_wc_ak'))
KP_wc_rk = Tools.load_from_file(os.path.join(knapsack_problem_path, 'KP_wc_rk'))
KP_sc_rk = Tools.load_from_file(os.path.join(knapsack_problem_path, 'KP_sc_rk'))
KP_uc_rk = Tools.load_from_file(os.path.join(knapsack_problem_path, 'KP_uc_rk'))

all_models = []

Tools.save_to_file(os.path.join(knapsack_problem_path, 'all_models'), all_models)

## KP_BGA

### KP_fitness

In [10]:
def knapsack_fitness_eval(population, problem, dims, pop):  
    fitness = np.zeros(pop)
    Weights = problem['w']
    Profits = problem['p']
    Ratios = Profits / Weights
    for i in range(pop):
        BV = population[i, :] == 1
        TotalWeight = np.sum(Weights[BV])
        TotalProfit = np.sum(Profits[BV])

        if TotalWeight > problem['cap']:  # Repair solution
            selections = np.sum(BV)
            List = np.zeros((selections, 2))
            counter = 0
            for j in range(dims):
                if BV[j] == 1:
                    List[counter, 0] = Ratios[j]
                    List[counter, 1] = int(j)
                    counter = counter + 1

                if counter >= selections:
                    break
            List = List[List[:, 0].argsort()[::-1]]
            counter = selections - 1
            while TotalWeight > problem['cap']:
                l = int(List[counter, 1])
                BV[l] = 0
                TotalWeight = TotalWeight - Weights[l]
                TotalProfit = TotalProfit - Profits[l]
                counter = counter - 1

        fitness[i] = TotalProfit
    return fitness

In [16]:
pop = 100
dims = 1000
population = np.round(np.random.rand(pop, dims))

fitness_uc_rk = knapsack_fitness_eval(population, KP_uc_rk, dims, pop)
print(fitness_uc_rk)
print(max(fitness_uc_rk))
fitness_sc_rk = knapsack_fitness_eval(population, KP_sc_rk, dims, pop)
print(fitness_sc_rk)
print(max(fitness_sc_rk))

[125. 137. 142. 130. 145. 138. 141. 156. 151. 147. 147. 141. 143. 131.
 150. 125. 159. 125. 149. 155. 140. 142. 151. 131. 146. 148. 128. 140.
 121. 158. 153. 134. 158. 133. 151. 139. 134. 150. 135. 150. 134. 123.
 138. 138. 143. 133. 137. 149. 155. 132. 143. 150. 130. 140. 134. 148.
 151. 147. 146. 143. 155. 136. 127. 138. 149. 138. 152. 148. 118. 149.
 134. 149. 134. 145. 133. 132. 139. 149. 153. 134. 156. 141. 145. 140.
 129. 127. 136. 148. 144. 135. 135. 132. 156. 134. 149. 132. 125. 149.
 162. 142.]
162.0
[120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120.
 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120.
 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120.
 120. 120. 120. 120. 120. 120. 120. 114. 120. 120. 120. 120. 120. 120.
 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120.
 120. 120. 120. 120. 120. 114. 120. 120. 114. 120. 115. 120. 120. 120.
 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120

In [17]:
from scipy.stats import multivariate_normal
class ProbabilityModel:  # Works reliably for 2(+) Dimensional distributions
    """ properties
        modeltype; % multivariate normal ('mvarnorm' - for real coded) or univariate marginal distribution ('umd' - for binary coded)    
        mean_noisy;
        mean_true;
        covarmat_noisy;
        covarmat_true;
        probofone_noisy;
        probofone_true;
        probofzero_noisy;
        probofzero_true;    
        vars;
      end"""

    #  methods (Static)
    def __init__(self, modeltype):
        self.modeltype = modeltype

    def buildmodel(self, solutions):
        pop, self.vars = solutions.shape
        self.probofone_true = np.mean(solutions, 0)
            # print(self.probofone_true)
        self.probofzero_true = 1 - self.probofone_true
            # print('probofone_true')
            # print(self.probofzero_true.shape)
        solutions_noisy = np.append(solutions, np.round(np.random.rand(round(0.1 * pop), self.vars)), axis=0)
            # print(solutions_noisy.shape)
        self.probofone_noisy = np.mean(solutions_noisy, 0)
            # print(self.probofone_noisy)
        self.probofzero_noisy = 1 - self.probofone_noisy
            # print(self.probofzero_noisy)


In [18]:
def KP_BGA(problem, dims, premSTOP, pop=100, gen=100, addr="problems/toy_problem"):
    """[bestSol, fitness_hist] = BGA(problem,dims,th_best): simple binary GA with
        uniform crossover and bit-flip mutation. 
        INPUT:
         problem: problem generated by GenKnapsack
         dims: problem dimensionality
         premSTOP: used for early stop if no improvement is made for 20
         consecutive generations
        
        OUTPUT:
         bestSol: best solution
         fitness: history of best fitness for each generation"""
    bestSol = None
    all_models = Tools.load_from_file(os.path.join(addr, 'all_models'))
    fitness_hist = np.zeros(gen)
    population = np.round(np.random.rand(pop, dims))
    fitness = knapsack_fitness_eval(population, problem, dims, pop)
    fitness_hist[0] = bestfitness = max(fitness)
    print('Generation 0 best fitness = ', str(fitness_hist[0]))

    counter = 0
    for i in range(1, gen):
        parent1 = population[np.random.permutation(pop), :]
        parent2 = population[np.random.permutation(pop), :]
        tmp = np.random.rand(pop, dims)
        offspring = np.zeros((pop, dims))
        index = tmp >= 0.5
        offspring[index] = parent1[index]
        index = tmp < 0.5
        offspring[index] = parent2[index]
        tmp = np.random.rand(pop, dims)
        index = tmp < (1 / dims)
        offspring[index] = np.abs(1 - offspring[index])
        cfitness = knapsack_fitness_eval(population, problem, dims, pop)
        interpop = np.append(population, offspring, 0)
        interfitness = np.append(fitness, cfitness)
        index = np.argsort((-interfitness))
        interfitness = interfitness[index]
        fitness = interfitness[:pop]
        interpop = interpop[index, :]
        population = interpop[:pop, :]
        fitness_hist[i] = fitness[0]
        print('Generation ', str(i), ' best fitness = ', str(fitness_hist[i]))

        if fitness[0] > bestfitness:
            bestfitness = fitness[0]
            counter = 0
        else:
            counter += 1
        if counter == 20 and premSTOP:
            fitness_hist[i:] = fitness_hist[i]
            break
    bestSol = population[0, :]
    model = ProbabilityModel('umd')
    model.buildmodel(population)
    all_models.append(model)
    Tools.save_to_file(os.path.join(addr, 'all_models'), all_models)

    return bestSol, fitness_hist

In [19]:
# build source probabilistic models
KP_BGA(KP_uc_rk, 1000, True, addr="problems/knapsack")
KP_BGA(KP_sc_rk, 1000, True, addr="problems/knapsack")
KP_BGA(KP_wc_rk, 1000, True, addr="problems/knapsack")
KP_BGA(KP_sc_ak, 1000, True, addr="problems/knapsack")

Generation 0 best fitness =  162.0
Generation  1  best fitness =  162.0
Generation  2  best fitness =  163.0
Generation  3  best fitness =  163.0
Generation  4  best fitness =  163.0
Generation  5  best fitness =  163.0
Generation  6  best fitness =  163.0
Generation  7  best fitness =  163.0
Generation  8  best fitness =  164.0
Generation  9  best fitness =  167.0
Generation  10  best fitness =  167.0
Generation  11  best fitness =  167.0
Generation  12  best fitness =  167.0
Generation  13  best fitness =  167.0
Generation  14  best fitness =  167.0
Generation  15  best fitness =  167.0
Generation  16  best fitness =  167.0
Generation  17  best fitness =  167.0
Generation  18  best fitness =  167.0
Generation  19  best fitness =  167.0
Generation  20  best fitness =  167.0
Generation  21  best fitness =  167.0
Generation  22  best fitness =  167.0
Generation  23  best fitness =  167.0
Generation  24  best fitness =  167.0
Generation  25  best fitness =  167.0
Generation  26  best fit

(array([0., 1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1.,
        1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0.,
        1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 1.,
        0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 0.,
        1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1.,
        0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
        0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0.,
        0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0.,
        1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 0.,
        0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1., 1., 0., 1.,
        0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1.,
        1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1.,
        1., 1., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0.,
        1., 